In [ ]:
f_func <- "../functions.ipynb"
eval(parse(text = system2('jupyter', c('nbconvert', f_func, '--to=script', '--stdout'), stdout = TRUE)))
f_path <- "../dirs.ipynb"
eval(parse(text = system2('jupyter', c('nbconvert', f_path, '--to=script', '--stdout'), stdout = TRUE)))

In [ ]:
master_table <- read.csv(file = paste0(INT_ReSeq, "classification_table.csv"), header = TRUE, row.names = 1) 

Asp_genus_count <- readRDS(paste0(INT_SPL, "Asp_genus_count.Rdata"))
Asp_species_count <- readRDS(paste0(INT_SPL, "Asp_species_count.Rdata"))

Asp_genus_RPM <- readRDS(paste0(INT_SPL, "Asp_genus_RPM.Rdata"))
Asp_species_RPM <- readRDS(paste0(INT_SPL, "Asp_species_RPM.Rdata"))

total_read_count <- readRDS(paste0(INT_SPL, "total_read_count.Rdata"))

In [ ]:
for (i in 1:length(Asp_genus_count)){
    print(paste(c(names(Asp_genus_count)[[i]]), ":", ncol(Asp_genus_count[[i]])))
}

In [ ]:
Bctrls <- colnames(Asp_genus_count[[1]])[which(grepl(colnames(Asp_genus_count[[1]]), pattern = "^H") & 
                                         grepl(colnames(Asp_genus_count[[1]]), pattern = "Bctrl$"))]

Pctrls <- colnames(Asp_genus_count[[1]])[which(grepl(colnames(Asp_genus_count[[1]]), pattern = "^H") & 
                                         grepl(colnames(Asp_genus_count[[1]]), pattern = "Pctrl$"))]

ctrl_list <- list(Bctrls, Pctrls)
names(ctrl_list) <- c("Bctrls", "Pctrls")

In [ ]:
print(paste0("mean # reads/patient sample: ", 
             round(mean(total_read_count[[1]][2, grepl(colnames(total_read_count[[1]]), pattern = "^A")], 
                        na.rm = TRUE))))

In [ ]:
MPM <- c(0.25,0.5,1,2,4,8,16,32,64,128,256,512,1024,2048,4096)
LOSD_species_table_withNoise <- master_table  %>% 
    filter(type == "TP species")  %>%
    filter(db != "RS") %>% 
    filter(!is.na(percentage)) %>% 
    mutate(threshold = sprintf("%.1f", as.numeric(gsub(threshold, pattern = "conf", replacement = "")))) %>% 
    filter(threshold %in% c("0.0","0.1","0.2","0.3","0.4","0.5","0.6","0.7","0.8","0.9","1.0")) %>% 
    filter(db %in% dbs_sel) %>% 
    filter(grepl(strain, pattern = "Aspergillus")) %>% 
    mutate(species = gsub(strain, pattern = "(?<=[a-z])([A-Z])", replacement = " ", perl = TRUE))  %>% 
    mutate(species = gsub(species, pattern = " .*", replacement = "")) %>% 
    mutate(species = gsub(species, pattern = "Aspergillus", replacement = "Aspergillus "))

for (i in 1:nrow(LOSD_species_table_withNoise)){
    tmp_db=LOSD_species_table_withNoise$db[i]
    tmp_fraction=LOSD_species_table_withNoise$percentage[i]/100
    tmp_specie=LOSD_species_table_withNoise$species[i]
    if (tmp_specie == "Aspergillus costaricaensis"){tmp_specie <- "Aspergillus costaricensis"}   
    tmp_conf=paste0("conf", LOSD_species_table_withNoise$threshold[i])
    tmp_threshold=LOSD_species_table_withNoise$threshold[i]
    
    
    for (NMR in c(10,25,40,55,70,85,100)){
        for (j in 1:2){         
            for (mpm in MPM){
                tmp_species_RPM <- Asp_species_RPM %>% 
                    filter(threshold == tmp_threshold) %>% 
                    filter(db == tmp_db) %>% 
                    mutate(RPM = as.numeric(RPM)) %>% 
                    filter(sample %in% ctrl_list[[j]]) %>% 
                    filter(taxName == tmp_specie) 
                
                nr_reads=round(c((NMR*mpm*tmp_fraction) + 
                            #add minimual noise observed in controls
                            (median(tmp_species_RPM$RPM)*NMR)), digits = 0) 
                nr_reads_total=(NMR*10^6)-nr_reads
            
                tmp <- c()
                for(bc in ctrl_list[[j]]){
                    tmp_species_count_bc <- Asp_species_count %>% 
                        filter(threshold == tmp_threshold) %>% 
                        filter(db == tmp_db) %>% 
                        mutate(count = as.numeric(count)) %>% 
                        filter(sample %in% bc) %>% 
                        filter(taxName == tmp_specie)

                    ft <- fisher.test(
                        rbind(c(nr_reads,nr_reads_total),
                                c(tmp_species_count_bc$count, 
                                  total_read_count[[tmp_conf]][tmp_db, bc]-tmp_species_count_bc$count)
                            ), 
                        alternative="greater")$p.value 

                    tmp <- c(tmp, ft)
                    rm(ft)
                }
                fisher_mean <- mean(tmp, na.rm = TRUE)
                if(fisher_mean < 0.001){
                    LOSD_species_table_withNoise[i, paste0("LOSD_", names(ctrl_list)[j], "_", NMR, "M")] <- mpm 
                    #print(mpm)
                    break
                } else if (mpm == MPM[length(MPM)]){
                    LOSD_species_table_withNoise[i, paste0("LOSD_", names(ctrl_list)[j], "_", NMR, "M")] <- paste0(">", MPM[length(MPM)])
                }
            }
        }
    }
    if (i%%5 == 0) {
        print(i)
        saveRDS(LOSD_species_table_withNoise, paste0(INT_LOSD, "LOSD_species_table_withNoise.Rdata"))
    }
}

In [ ]:
MPM <- c(0.25,0.5,1,2,4,8,16,32,64,128,256,512,1024,2048,4096)
LOSD_species_table_withNoise <- master_table  %>% 
    filter(type == "TP species")  %>%
    filter(db != "RS") %>% 
    filter(!is.na(percentage)) %>% 
    mutate(threshold = sprintf("%.1f", as.numeric(gsub(threshold, pattern = "conf", replacement = "")))) %>% 
    filter(threshold %in% c("0.0","0.1","0.2","0.3","0.4","0.5","0.6","0.7","0.8","0.9","1.0")) %>% 
    filter(db %in% dbs_sel) %>% 
    filter(grepl(strain, pattern = "Aspergillus")) %>% 
    mutate(species = gsub(strain, pattern = "(?<=[a-z])([A-Z])", replacement = " ", perl = TRUE))  %>% 
    mutate(species = gsub(species, pattern = " .*", replacement = "")) %>% 
    mutate(species = gsub(species, pattern = "Aspergillus", replacement = "Aspergillus "))

for (i in 1:nrow(LOSD_species_table_withNoise)){
    tmp_db=LOSD_species_table_withNoise$db[i]
    tmp_fraction=LOSD_species_table_withNoise$percentage[i]/100
    tmp_specie=LOSD_species_table_withNoise$species[i]
    if (tmp_specie == "Aspergillus costaricaensis"){tmp_specie <- "Aspergillus costaricensis"}   
    tmp_conf=paste0("conf", LOSD_species_table_withNoise$threshold[i])
    tmp_threshold=LOSD_species_table_withNoise$threshold[i]
    
    
    for (NMR in c(10,25,40,55,70,85,100)){
        for (j in 1:2){         
            for (mpm in MPM){
                tmp_species_RPM <- Asp_species_RPM %>% 
                    filter(threshold == tmp_threshold) %>% 
                    filter(db == tmp_db) %>% 
                    mutate(RPM = as.numeric(RPM)) %>% 
                    filter(sample %in% ctrl_list[[j]]) %>% 
                    filter(taxName == tmp_specie) 
                
                nr_reads=round(c((NMR*mpm*tmp_fraction) + 
                            #add minimual noise observed in controls
                            (median(tmp_species_RPM$RPM)*NMR)), digits = 0) 
                nr_reads_total=(NMR*10^6)-nr_reads
            
                tmp <- c()
                for(bc in ctrl_list[[j]]){
                    tmp_species_count_bc <- Asp_species_count %>% 
                        filter(threshold == tmp_threshold) %>% 
                        filter(db == tmp_db) %>% 
                        mutate(count = as.numeric(count)) %>% 
                        filter(sample %in% bc) %>% 
                        filter(taxName == tmp_specie)

                    ft <- fisher.test(
                        rbind(c(nr_reads,nr_reads_total),
                                c(tmp_species_count_bc$count, 
                                  total_read_count[[tmp_conf]][tmp_db, bc]-tmp_species_count_bc$count)
                            ), 
                        alternative="greater")$p.value 

                    tmp <- c(tmp, ft)
                    rm(ft)
                }
                fisher_mean <- mean(tmp, na.rm = TRUE)
                if(fisher_mean < 0.001){
                    LOSD_species_table_withNoise[i, paste0("LOSD_", names(ctrl_list)[j], "_", NMR, "M")] <- mpm 
                    #print(mpm)
                    break
                } else if (mpm == MPM[length(MPM)]){
                    LOSD_species_table_withNoise[i, paste0("LOSD_", names(ctrl_list)[j], "_", NMR, "M")] <- paste0(">", MPM[length(MPM)])
                }
            }
        }
    }
    if (i%%5 == 0 | i == nrow(LOSD_species_table_withNoise)) {
        print(i)
        saveRDS(LOSD_species_table_withNoise, paste0(INT_LOSD, "LOSD_species_table_withNoise.Rdata"))
    }
}